In [1]:
#import dependencies
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#create a df
list_of_columns = ["appid","Name","Developer","Publisher","Owners","Metascore","Genre(s)"]
steam_spy_df = pd.DataFrame(columns=list_of_columns)

#steam spy blocks automated web scraping so we need to pass it a header telling it that we are a human using a regular web browser
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

#we need this later when looping through all genres
stop = BeautifulSoup('''<br/>''', 'html.parser')
stop_point = stop.find("br")

In [ ]:
u = 1

#loop through all steam appids?
while u <= 1500000:
    #print to screen what appid it's trying so you know it didn't crash
    print("Trying appid #" + str(u))
    url = "https://steamspy.com/app/" + str(u)
    r = requests.get(url, headers=header)
    soup = BeautifulSoup(r.text, 'html.parser')
    #pull title
    title_raw = soup.find("h3")
    title = title_raw.get_text()
    #if title is blank, don't save it to the DF
    if title == "":
        u = u + 1
    else:
        try:
            #find developer
            developer = soup.find(text="Developer:")
            developer_next = developer.parent.findNext()
            developer_text = developer_next.get_text()
            #find publisher
            publisher = soup.find(text="Publisher:")
            publisher_next = publisher.parent.findNext()
            publisher_text = publisher_next.get_text()
            #find metascore
            metascore = soup.find('strong', text="Metascore:")
            metascore_next = metascore.next_sibling.strip()
            #find owners
            owners = soup.find('strong', text="Owners")
            owners_next = owners.next_sibling.strip()
            owners_next = owners_next[2:]
            #find genres
            genre = soup.find('strong', text="Genre:")
            genre_parent = genre.parent()
            i = 13
            end_of_line = False
            list_of_genres = []
            while end_of_line == False:
                if genre_parent[i] == stop_point:
                    end_of_line = True
                else:
                    list_of_genres.append(genre_parent[i].get_text())
                    i = i+1
            #save to df
            steam_spy_df.append({"appid": u,
                                 "Name": title,
                                 "Developer": developer_text,
                                 "Publisher": publisher_text,
                                 "Owners": owners_next,
                                 "Metascore": metascore_next,
                                 "Genre(s)": list_of_genres}, ignore_index=True)
            u = u + 1
        except AttributeError:
            print("Woops!")
            u = u + 1

Trying appid #1
Trying appid #2
Trying appid #3
Trying appid #4
Trying appid #5


In [ ]:
steam_spy_df